# Cut Fill Analysis

Cut-fill analysis is a technique used in earthwork and construction projects to determine the volume of material that needs to be cut (removed) or filled (added) at a particular location. It is commonly performed in grading and excavation projects, such as road construction, building construction, and land development.

The main objective of cut-fill analysis is to assess the balance between the earthwork cut and fill quantities to achieve desired topography or grading requirements. It helps in optimizing the project design, estimating material quantities, and managing project costs.

Cut-fill analysis provides valuable information for project planning, cost estimation, and earthwork management. It helps in optimizing the design to minimize excessive cut or fill, balancing the excavation and fill quantities to reduce material transportation, and ensuring the stability and integrity of the constructed features.

In [ ]:

from osgeo import gdal, osr
import numpy as np

In [ ]:
# Calculate area and volume above the base height
def cut_area_volume(dem_data: np.ndarray, base_height: float, cell_size: float):
  binary_mask = np.where(dem_data > base_height, 1, 0) # Create binary mask of DEM
  cut_data = dem_data[dem_data > base_height]
  cut_area = cell_size * cut_data.shape[0]
  cut_volume = np.sum((dem_data - base_height) * binary_mask * cell_size)
  return cut_area, cut_volume

In [ ]:
# Calculate area and volume below the base height
def fill_area_volume(dem_data: np.ndarray, base_height: float, cell_size: float):
  binary_mask = np.where(dem_data > base_height, 0, 1) # Create binary mask of DEM
  fill_data = dem_data[dem_data < base_height]
  fill_area = cell_size * fill_data.shape[0]
  fill_volume = np.sum((base_height - dem_data) * binary_mask * cell_size)
  return fill_area, fill_volume

In [ ]:
# Load DEM using gdal
ds = gdal.Open('clipped_raster.tif')
dem_band = ds.GetRasterBand(1)
dem_data = dem_band.ReadAsArray()
dem_data = dem_data.flatten() # Convert to 1D array

# Determine height at which to calculate cut and fill volumes # We can also take user input
base_height = 3 #elevation_average 

# Remove nodata value from the raster
nodata = ds.GetRasterBand(1).GetNoDataValue()
dem_data = np.delete(dem_data, np.argwhere(dem_data == nodata))

# Get geotransform and projection of DEM
geotransform = ds.GetGeoTransform()
projection = ds.GetProjection()

# Check if dataset is in projected or geographic coordinate system
cs = osr.SpatialReference()
cs.ImportFromWkt(projection)

if cs.IsProjected():
    cell_size = abs(geotransform[1] * geotransform[5])
else:
    print('Dataset is in geographic coordinate system, area and volume calculation will not be precise')
    cell_size = abs(geotransform[1]*111320.0 * geotransform[5]*111320.0) # Convet geographic distance to meter


cut_area, cut_volume = cut_area_volume(dem_data, base_height, cell_size)
fill_area, fill_volume = fill_area_volume(dem_data, base_height, cell_size)
net_volume = cut_volume - fill_volume

print(f"Cut area at {base_height} meter is {cut_area} square meters, and volume is {cut_volume} cubic meters")
print(f"Fill area at {base_height} meter is {fill_area} square meters, and volume is {fill_volume} cubic meters")
print(f"Net cut-fill volume at {base_height} meter is {net_volume} cubic meters") # +ve sign for cut and -ve for fill

# Close dataset
ds = None

In [ ]:
## Finding the optimum elevation at which there is minimum cut and fill

def compute_cut_fill_volumes(data, elev, cell_size):
    # Calculate the cut and fill volumes for the given elevation
    cut_area, cut_volume = cut_area_volume(data, elev, cell_size)
    fill_area, fill_volume = fill_area_volume(data, elev, cell_size)
    return cut_volume, fill_volume

def compute_optimal_elevation(dem):
    # Get the raster band and read the data as a NumPy array
    band = dem.GetRasterBand(1)
    data = band.ReadAsArray()
    data = data.flatten()
    nodata = dem.GetRasterBand(1).GetNoDataValue()
    data = np.delete(data, np.argwhere(data == nodata))

    (min, max, mean, std) = band.GetStatistics(True, True)

    geotransform = dem.GetGeoTransform()
    cell_size = abs(geotransform[1] * geotransform[5])

    # Set the initial lower and upper bounds for the elevation range
    lower_bound = min
    upper_bound = max

    # Set the initial tolerance for the elevation range
    tolerance = 0.0001

    # Iterate until the difference between the upper and lower bounds is less than the tolerance
    while upper_bound - lower_bound > tolerance:
        # Calculate the midpoint of the elevation range
        midpoint_elev = (lower_bound + upper_bound) / 2

        # Compute the cut and fill volumes for the elevation midpoint
        cut_volume, fill_volume = compute_cut_fill_volumes(data, midpoint_elev, cell_size)

        # If the cut volume is greater than the fill volume, set the new lower bound to the midpoint elevation
        if cut_volume > fill_volume:
            lower_bound = midpoint_elev
        # Otherwise, set the new upper bound to the midpoint elevation
        else:
            upper_bound = midpoint_elev

    # The optimal elevation for cut-fill analysis is the midpoint elevation of the final elevation range
    optimal_elev = (lower_bound + upper_bound) / 2
    return optimal_elev

# Open the DEM file using GDAL
dem = gdal.Open('clipped_raster.tif')
optimum_elevation = compute_optimal_elevation(dem)
print(f"The optimum elevation for minimum cut fill is: {optimum_elevation}")